In [1]:
# configure auto-reload and add libs to path
%load_ext autoreload
%autoreload 2

import os, sys
fastai_lib_path = os.path.abspath('../../..')
if fastai_lib_path not in sys.path: sys.path.append(fastai_lib_path)

In [2]:
# core imports
import json, requests, ast

from fastai.imports import *
from fastai.structured import *

from pandas_summary import DataFrameSummary
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier

from sklearn import metrics

In [3]:
# pandas and plotting config
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)

%matplotlib inline

Configure path and TBA Read API key (which is required to use v3 of their API).

In [4]:
PATH = 'data'

comp_yr = 2017

tba_key = 'J9XyDSN69eZMHqanEDaevzAywDjxL9iyBYAQh2erLJJ9MtZVG60HsEYeKVwloFGe'
tba_header = { 'X-TBA-Auth-Key': tba_key }
tba_url = 'https://thebluealliance.com/api/v3'

team_match_scores_path = f'{PATH}/{comp_yr}-team-match-scores-raw.csv'
team_event_stats_path = f'{PATH}/{comp_yr}-team-event-stats-raw.csv'
scouting_reports_path = f'{PATH}/{comp_yr}-scouting-reports-raw.csv'

In [5]:
def replace_dict(df, col, default_val=''):
    new_df = pd.DataFrame((ast.literal_eval(v) for k,v in df[col].iteritems() if (str(v) != 'nan')))
    new_df.columns = [f'{col}_{c}' for c in new_df.columns]
    ret = pd.concat([df,new_df.fillna(default_val)], axis=1)
    
    del ret[col]
    return ret

## 1. Prep the scouting reports

In [6]:
scouting_reports_df = pd.read_csv(scouting_reports_path, low_memory=False)
scouting_reports_df.head()

,asdf,dddd,event_id,rating_overall_gear_efficiency,rating_overall_gear_efficiency_auto,rating_overall_gear_placement_auto,rating_overall_pilot_competency,rating_overall_robot_stability,rating_scoring_airship_climb,rating_scoring_base_line_made_auto,rating_scoring_gears_made,rating_scoring_gears_made_auto,rating_scoring_high_goals_made,rating_scoring_high_goals_made_auto,rating_scoring_low_goals_made,rating_scoring_low_goals_made_auto,scored_at,scored_by,team_id
0,NaN,NaN,2017cada,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,1.490371e+12,"{'email': 'tylerselinka1@gmail.com', 'name': '...",frc3013
1,NaN,NaN,2017cada,2.0,NaN,0.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.490371e+12,"{'email': 'connorrocker@gmail.com', 'name': 'C...",frc2551
2,NaN,NaN,2017cada,2.0,NaN,0.0,2.0,2.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.490371e+12,"{'email': 'brian.titcomb27@gmail.com', 'name':...",frc692
3,NaN,NaN,2017cada,4.0,NaN,0.0,NaN,4.0,1.0,NaN,3.0,0.0,0.0,0.0,0.0,0.0,1.490371e+12,"{'email': 'seanlinden7934@gmail.com', 'name': ...",frc1323
4,NaN,NaN,2017cada,2.0,NaN,0.0,4.0,3.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.490371e+12,"{'email': 'sca2019@gmail.com', 'name': 'Seth A...",frc4698


In [7]:
scouting_reports_df = replace_dict(scouting_reports_df, 'scored_by')

In [8]:
# drop the 'scored_by' column (and other weird/probably test values)
scouting_reports_df.drop(['asdf', 'dddd'], axis=1, inplace=True)

In [9]:
# remove rows without an event_id
scouting_reports_df = scouting_reports_df[pd.notnull(scouting_reports_df['event_id'])]

In [10]:
# convert all NaNs to 0
scouting_reports_df.fillna(0, inplace=True)

In [11]:
scouting_reports_df.rename(columns={'event_id': 'event_key', 'team_id': 'team_key'}, inplace=True)

In [12]:
# convert unix timestamps to datetime
scouting_reports_df['scored_at'] = pd.to_datetime(scouting_reports_df['scored_at'], unit='ms')

# sort by date ascending (most recent on top)
scouting_reports_df.sort_values('scored_at', ascending=False, inplace=True)

In [13]:
d = DataFrameSummary(scouting_reports_df)
# d.columns_stats.transpose()
d.summary().transpose()

,count,mean,std,min,25%,50%,75%,max,counts,uniques,missing,missing_perc,types
event_key,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1234,4,0,0%,categorical
rating_overall_gear_efficiency,1234,2.36467,1.64977,0,1,3,4,5,1234,6,0,0%,numeric
rating_overall_gear_efficiency_auto,1234,0.871151,1.61916,0,0,0,1,5,1234,6,0,0%,numeric
rating_overall_gear_placement_auto,1234,0.0794165,0.270497,0,0,0,0,1,1234,2,0,0%,bool
rating_overall_pilot_competency,1234,1.62804,2.07996,0,0,0,4,5,1234,6,0,0%,numeric
rating_overall_robot_stability,1234,3.19773,1.2843,0,3,3,4,5,1234,6,0,0%,numeric
rating_scoring_airship_climb,1234,0.560778,0.496494,0,0,1,1,1,1234,2,0,0%,bool
rating_scoring_base_line_made_auto,1234,0.258509,0.437993,0,0,0,1,1,1234,2,0,0%,bool
rating_scoring_gears_made,1234,1.52917,1.29051,0,0,1,2,6,1234,7,0,0%,numeric
rating_scoring_gears_made_auto,1234,0.197731,0.40651,0,0,0,0,3,1234,4,0,0%,numeric


## 2. Prep the match scores

In [14]:
matches_df = pd.read_csv(team_match_scores_path, low_memory=False)
matches_df.head()

,actual_time,adjustPoints,alliance,alliance_team_keys,autoFuelHigh,autoFuelLow,autoFuelPoints,autoMobilityPoints,autoPoints,autoRotorPoints,comp_level,event_city,event_code,event_country,event_district,event_end_date,event_key,event_name,event_start_date,event_state_prov,event_type,foulCount,foulPoints,is_winner,kPaBonusPoints,kPaRankingPointAchieved,match_key,match_number,post_result_time,predicted_time,robot1Auto,robot2Auto,robot3Auto,rotor1Auto,rotor1Engaged,rotor2Auto,rotor2Engaged,rotor3Engaged,rotor4Engaged,rotorBonusPoints,rotorRankingPointAchieved,score,set_number,tba_rpEarned,team_key,techFoulCount,teleopFuelHigh,teleopFuelLow,teleopFuelPoints,teleopPoints,teleopRotorPoints,teleopTakeoffPoints,time,totalPoints,touchpadFar,touchpadMiddle,touchpadNear,winning_margin,year
0,1488551305,0,blue,"['frc3651', 'frc6693']",0,0,0,5,5,0,qm,Myrtle Beach,scmb,USA,NaN,2017-03-04,2017scmb,Palmetto Regional,2017-03-01,SC,0,0,30,1,0,False,2017scmb_qm4,4,1.488552e+09,1.488552e+09,Mobility,None,None,False,True,False,False,False,False,0,False,75,1,2.0,frc1287,0,0,0,0,40,40,0,1488551040,75,None,None,None,30,2017
1,1488551818,0,red,"['frc3653', 'frc2152']",0,0,0,15,15,0,qm,West Palm Beach,flwp,USA,NaN,2017-03-04,2017flwp,South Florida Regional,2017-03-01,FL,0,0,0,1,0,False,2017flwp_qm1,1,1.488552e+09,1.488552e+09,Mobility,Mobility,Mobility,False,True,False,False,False,False,0,False,155,1,2.0,frc6038,0,2,0,0,140,40,100,1488551400,155,None,ReadyForTakeoff,ReadyForTakeoff,44,2017
2,1488552377,0,blue,"['frc5196', 'frc6388']",0,0,0,0,0,0,qm,West Palm Beach,flwp,USA,NaN,2017-03-04,2017flwp,South Florida Regional,2017-03-01,FL,0,0,0,0,0,False,2017flwp_qm2,2,1.488553e+09,1.488553e+09,None,None,None,False,True,False,True,False,False,0,False,80,1,0.0,frc263,0,0,0,0,80,80,0,1488551820,80,None,None,None,-45,2017
3,1488552377,0,red,"['frc694', 'frc2641']",0,0,0,5,5,0,qm,West Palm Beach,flwp,USA,NaN,2017-03-04,2017flwp,South Florida Regional,2017-03-01,FL,0,0,0,1,0,False,2017flwp_qm2,2,1.488553e+09,1.488553e+09,Mobility,None,None,False,True,False,True,True,False,0,False,125,1,2.0,frc1523,0,0,0,0,120,120,0,1488551820,125,None,None,None,45,2017
4,1488553596,0,blue,"['frc4823', 'frc6366']",0,0,0,10,10,0,qm,Myrtle Beach,scmb,USA,NaN,2017-03-04,2017scmb,Palmetto Regional,2017-03-01,SC,0,0,25,1,0,False,2017scmb_qm8,8,1.488554e+09,1.488554e+09,None,Mobility,Mobility,False,True,False,True,False,False,0,False,165,1,2.0,frc4451,0,0,0,0,130,80,50,1488552960,165,None,None,ReadyForTakeoff,10,2017


In [15]:
# map autonomous to 1 or 0
print(matches_df.robot1Auto.unique(), matches_df.robot2Auto.unique(), matches_df.robot3Auto.unique())
robotAutoMap = { 'Mobility': 1, 'None': 0, 'Unknown': 0 }

matches_df.robot1Auto = matches_df.robot1Auto.map(robotAutoMap)
matches_df.robot2Auto = matches_df.robot2Auto.map(robotAutoMap)
matches_df.robot3Auto = matches_df.robot3Auto.map(robotAutoMap)

['Mobility' 'None'] ['None' 'Mobility'] ['None' 'Mobility']


In [16]:
# map takeoffs to 1 or 0
print(matches_df.touchpadFar.unique(), matches_df.touchpadMiddle.unique(), matches_df.touchpadNear.unique())
touchpadMap = { 'ReadyForTakeoff': 1, 'None': 0, 'Unknown': 0 }

matches_df.touchpadFar = matches_df.touchpadFar.map(touchpadMap)
matches_df.touchpadMiddle = matches_df.touchpadMiddle.map(touchpadMap)
matches_df.touchpadNear = matches_df.touchpadNear.map(touchpadMap)

['None' 'ReadyForTakeoff'] ['None' 'ReadyForTakeoff'] ['None' 'ReadyForTakeoff']


In [17]:
# convert boolean columns and those that maintain a count to int
int_cols = [
    'rotor1Auto', 'rotor1Engaged', 'rotor2Auto', 
    'rotor2Engaged', 'rotor3Engaged', 'rotor4Engaged',  
]

matches_df[int_cols] = matches_df[int_cols].astype(int)

In [18]:
# create additional columns that aggregate others
matches_df['robotAutoCount'] = matches_df['robot1Auto'] + matches_df['robot2Auto'] + matches_df['robot3Auto']
matches_df['rotorAutoCount'] = matches_df['rotor1Auto'] + matches_df['rotor2Auto']
matches_df['rotorEngagedCount'] = matches_df['rotor1Engaged'] + matches_df['rotor2Engaged'] + matches_df['rotor3Engaged'] + matches_df['rotor4Engaged']
matches_df['takeoffCount'] = matches_df['touchpadFar'] + matches_df['touchpadMiddle'] + matches_df['touchpadNear']

In [19]:
# convert unix timestamps to datetime
datetime_cols = ['actual_time', 'post_result_time', 'predicted_time', 'time']
matches_df[datetime_cols] = matches_df[datetime_cols].apply(pd.to_datetime, unit='s')

# sort data by even start_date (most recent on top)
matches_df.sort_values('actual_time', ascending=False, inplace=True)

In [20]:
# convert NaNs to 0
matches_df.fillna(0.0, inplace=True)

In [21]:
d = DataFrameSummary(matches_df)
# d.columns_stats.transpose()
d.summary().transpose()

,count,mean,std,min,25%,50%,75%,max,counts,uniques,missing,missing_perc,types
actual_time,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5417,2756,0,0%,date
adjustPoints,5417,-0.2566,9.82813,-445,0,0,0,0,5417,4,0,0%,numeric
alliance,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5417,2,0,0%,bool
alliance_team_keys,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5417,4541,0,0%,categorical
autoFuelHigh,5417,3.03397,7.23439,0,0,0,2,56,5417,51,0,0%,numeric
autoFuelLow,5417,0.137161,1.04683,0,0,0,0,10,5417,11,0,0%,numeric
autoFuelPoints,5417,3.07218,7.22559,0,0,0,3,56,5417,51,0,0%,numeric
autoMobilityPoints,5417,13.5352,2.77483,0,15,15,15,15,5417,4,0,0%,numeric
autoPoints,5417,49.105,32.9633,0,15,70,75,166,5417,123,0,0%,numeric
autoRotorPoints,5417,32.4977,30.7966,0,0,60,60,120,5417,3,0,0%,numeric


## 3. Prep event stats

In [22]:
stats_df = pd.read_csv(team_event_stats_path, low_memory=False)
stats_df.head()

,Auto,Match Points,Pressure,Ranking Score,Rotor,Touchpad,ccwms,dprs,event_key,is_finals_team,is_playoff_team,losses,matches_played,oprs,post_qual_wins,qual_average,rank,start_date,team_key,ties,wins
0,339.0,1536.0,6.0,1.50,800.0,550.0,34.356540,44.906465,2017scmb,1,1,2,8,79.263005,5,NaN,6,2017-03-01,frc1287,0,6
1,165.0,1513.0,3.0,1.37,780.0,600.0,31.595175,64.709658,2017scmb,0,1,2,8,96.304833,1,NaN,11,2017-03-01,frc4451,1,5
2,228.0,1453.0,8.0,1.25,760.0,550.0,23.209025,55.120927,2017scmb,1,1,3,8,78.329953,5,NaN,17,2017-03-01,frc283,0,5
3,335.0,1974.0,24.0,1.60,1040.0,700.0,48.955095,48.207602,2017flwp,1,1,2,10,97.162697,5,NaN,3,2017-03-01,frc1523,0,8
4,326.0,1793.0,23.0,1.20,1040.0,550.0,24.413285,52.241866,2017flwp,0,1,4,10,76.655151,2,NaN,15,2017-03-01,frc263,0,6


In [23]:
# convert whatever columns can to be numeric
stats_df = stats_df.apply(pd.to_numeric, errors='ignore')

In [24]:
# convert NaNs to 0
stats_df.fillna(0, inplace=True)

In [25]:
# sort data by even start_date (most recent on top)
stats_df.sort_values('start_date', ascending=False, inplace=True)

In [26]:
d = DataFrameSummary(stats_df)
# d.columns_stats.transpose()
d.summary().transpose()

,count,mean,std,min,25%,50%,75%,max,counts,uniques,missing,missing_perc,types
Auto,437,428.419,180.639,91,292,421,555,988,437,322,0,0%,numeric
Match Points,437,2227.12,639.44,825,1745,2220,2752,3889,437,392,0,0%,numeric
Pressure,437,47.6018,75.0894,0,6,26,58,577,437,126,0,0%,numeric
Ranking Score,437,1.21613,0.447002,0,0.9,1.2,1.5,2.8,437,76,0,0%,numeric
Rotor,437,1115.88,309.29,460,860,1100,1380,1820,437,67,0,0%,numeric
Touchpad,437,854.577,304.499,50,600,900,1100,1650,437,32,0,0%,numeric
ccwms,437,8.70183,35.1521,-99.4342,-14.0945,9.55658,31.5952,111.992,437,437,0,0%,numeric
dprs,437,72.2131,26.442,6.92773,54.5413,72.7121,88.0207,153.803,437,437,0,0%,numeric
event_key,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,437,67,0,0%,categorical
is_finals_team,437,0.19222,0.394496,0,0,0,0,1,437,2,0,0%,bool


## 4. Combine datasets

In [37]:
scouting_by_team_df = scouting_reports_df.groupby(by=['team_key', 'event_key']).agg('mean')
matches_by_team_df = matches_df[matches_df.comp_level == 'qm'].groupby(by=['team_key', 'event_key']).agg('mean')
stats_by_team_df = stats_df.groupby(by=['team_key', 'event_key']).agg('mean')

In [38]:
combined_df = pd.concat([scouting_by_team_df, matches_by_team_df, stats_by_team_df], axis=1, join='inner')

In [42]:
combined_df.head()

,,rating_overall_gear_efficiency,rating_overall_gear_efficiency_auto,rating_overall_gear_placement_auto,rating_overall_pilot_competency,rating_overall_robot_stability,rating_scoring_airship_climb,rating_scoring_base_line_made_auto,rating_scoring_gears_made,rating_scoring_gears_made_auto,rating_scoring_high_goals_made,rating_scoring_high_goals_made_auto,rating_scoring_low_goals_made,rating_scoring_low_goals_made_auto,adjustPoints,autoFuelHigh,autoFuelLow,autoFuelPoints,autoMobilityPoints,autoPoints,autoRotorPoints,event_type,foulCount,foulPoints,is_winner,kPaBonusPoints,kPaRankingPointAchieved,match_number,robot1Auto,robot2Auto,robot3Auto,rotor1Auto,rotor1Engaged,rotor2Auto,rotor2Engaged,rotor3Engaged,rotor4Engaged,rotorBonusPoints,rotorRankingPointAchieved,score,set_number,tba_rpEarned,techFoulCount,teleopFuelHigh,teleopFuelLow,teleopFuelPoints,teleopPoints,teleopRotorPoints,teleopTakeoffPoints,totalPoints,touchpadFar,touchpadMiddle,touchpadNear,winning_margin,year,robotAutoCount,rotorAutoCount,rotorEngagedCount,takeoffCount,Auto,Match Points,Pressure,Ranking Score,Rotor,Touchpad,ccwms,dprs,is_finals_team,is_playoff_team,losses,matches_played,oprs,post_qual_wins,qual_average,rank,ties,wins
team_key,event_key,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
frc100,2017cada,1.222222,0.000000,0.000000,2.222222,3.222222,0.777778,0.666667,0.777778,0.000000,0.000000,0.0,0.0,0.0,0.0,2.333333,0.777778,2.555556,13.888889,43.111111,26.666667,0.0,0.222222,1.111111,0.333333,0.0,0.111111,42.222222,1.000000,0.888889,0.888889,0.444444,1.0,0.0,1.000000,0.111111,0.0,0.0,0.0,198.555556,1.0,0.777778,0.777778,13.222222,0.333333,4.333333,154.333333,66.666667,83.333333,198.555556,0.666667,0.333333,0.666667,-22.777778,2017.0,2.777778,0.444444,2.111111,1.666667,388.0,1787.0,62.0,0.77,840.0,750.0,-6.638987,71.136739,0,0,6,9,64.497752,0,0.0,44,0,3
frc1056,2017cada,2.818182,0.818182,0.181818,2.818182,3.363636,0.727273,0.000000,1.636364,0.181818,0.000000,5.0,0.0,0.0,0.0,9.000000,0.000000,9.000000,13.888889,56.222222,33.333333,0.0,0.222222,1.111111,0.666667,0.0,0.111111,42.555556,1.000000,0.888889,0.888889,0.555556,1.0,0.0,0.888889,0.222222,0.0,0.0,0.0,229.666667,1.0,1.444444,0.555556,13.666667,0.777778,4.555556,172.333333,62.222222,105.555556,229.666667,0.888889,0.555556,0.666667,35.111111,2017.0,2.777778,0.555556,2.111111,2.111111,506.0,2067.0,122.0,1.44,860.0,950.0,11.418404,63.450057,0,1,3,9,74.868461,1,0.0,8,0,6
frc1072,2017cada,3.400000,1.400000,0.100000,1.700000,3.400000,0.700000,0.200000,1.900000,0.100000,1.000000,2.0,0.0,0.5,0.0,2.222222,0.000000,2.222222,13.333333,35.555556,20.000000,0.0,0.111111,5.555556,0.555556,0.0,0.000000,43.222222,0.888889,1.000000,0.777778,0.333333,1.0,0.0,1.000000,0.555556,0.0,0.0,0.0,214.222222,1.0,1.111111,0.777778,3.000000,0.000000,0.888889,173.111111,88.888889,83.333333,214.222222,0.666667,0.444444,0.555556,-12.111111,2017.0,2.666667,0.333333,2.555556,1.666667,320.0,1928.0,28.0,1.11,980.0,750.0,-14.813528,78.854740,0,1,4,9,64.041212,2,0.0,27,0,5
frc108,2017tur,4.181818,1.636364,0.272727,3.909091,4.272727,0.909091,0.181818,3.000000,0.363636,0.090909,0.0,0.0,0.0,0.0,1.500000,0.000000,1.500000,14.500000,46.000000,30.000000,3.0,0.100000,31.500000,0.500000,0.0,0.000000,54.800000,1.000000,0.900000,1.000000,0.500000,1.0,0.0,1.000000,1.000000,0.5,0.0,0.5,308.600000,1.0,1.500000,0.000000,3.700000,0.000000,1.100000,231.100000,120.000000,110.000000,308.600000,0.700000,0.900000,0.600000,34.400000,2017.0,2.900000,0.500000,3.500000,2.200000,460.0,3086.0,26.0,1.50,1500.0,1100.0,38.493471,87.810154,0,1,5,10,126.303625,0,0.0,23,0,5
frc1165,2017tur,1.750000,0.000000,0.000000,1.000000,3.250000,1.000000,0.000000,1.500000,0.250000,0.000000,0.0,0.0,0.0,0.0,2.900000,0.000000,2.900000,15.000000,47.900000,30.000000,3.0,0.200000,1.000000,0.300000,0.0,0.100000,55.400000,1.000000,1.000000,1.000000,0.500000,1.0,0.0,1.000000,0.900000,0.4,0.0,0.4,278.000000,1.0,1.100000,0.300000,6.700000,0.600000,2.100000,229.100000,112.

In [46]:
combined_df['wining_ratio'] = combined_df.wins / combined_df.matches_played

In [47]:
combined_df.loc['frc2102']

,rating_overall_gear_efficiency,rating_overall_gear_efficiency_auto,rating_overall_gear_placement_auto,rating_overall_pilot_competency,rating_overall_robot_stability,rating_scoring_airship_climb,rating_scoring_base_line_made_auto,rating_scoring_gears_made,rating_scoring_gears_made_auto,rating_scoring_high_goals_made,rating_scoring_high_goals_made_auto,rating_scoring_low_goals_made,rating_scoring_low_goals_made_auto,adjustPoints,autoFuelHigh,autoFuelLow,autoFuelPoints,autoMobilityPoints,autoPoints,autoRotorPoints,event_type,foulCount,foulPoints,is_winner,kPaBonusPoints,kPaRankingPointAchieved,match_number,robot1Auto,robot2Auto,robot3Auto,rotor1Auto,rotor1Engaged,rotor2Auto,rotor2Engaged,rotor3Engaged,rotor4Engaged,rotorBonusPoints,rotorRankingPointAchieved,score,set_number,tba_rpEarned,techFoulCount,teleopFuelHigh,teleopFuelLow,teleopFuelPoints,teleopPoints,teleopRotorPoints,teleopTakeoffPoints,totalPoints,touchpadFar,touchpadMiddle,touchpadNear,winning_margin,year,robotAutoCount,rotorAutoCount,rotorEngagedCount,takeoffCount,Auto,Match Points,Pressure,Ranking Score,Rotor,Touchpad,ccwms,dprs,is_finals_team,is_playoff_team,losses,matches_played,oprs,post_qual_wins,qual_average,rank,ties,wins,win_ratio,wining_ratio
event_key,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2017cada,2.285714,0.142857,0.000000,4.285714,2.857143,0.285714,0.142857,1.571429,0.000000,3.285714,5.714286,0.0,0.0,0.0,5.333333,0.0,5.333333,12.777778,38.111111,20.0,0.0,0.00,7.222222,0.444444,0.0,0.0,39.555556,0.888889,0.666667,1.00,0.333333,1.0,0.0,1.0,0.111111,0.0,0.0,0.0,186.111111,1.0,0.888889,0.000,9.666667,0.111111,3.0,140.777778,71.111111,66.666667,186.111111,0.444444,0.333333,0.555556,-24.888889,2017.0,2.555556,0.333333,2.111111,1.333333,343.0,1675.0,75.0,0.88,820.0,600.0,-22.039435,63.147163,0,1,5,9,41.107728,1,0.0,40,0,4,0.444444,0.444444
2017casd,2.666667,1.833333,0.000000,0.500000,3.000000,0.333333,0.000000,1.500000,0.333333,7.500000,3.333333,0.0,0.0,0.0,1.375000,0.0,1.375000,13.125000,44.500000,30.0,0.0,0.25,12.500000,0.875000,0.0,0.0,42.500000,0.875000,1.000000,0.75,0.500000,1.0,0.0,1.0,0.625000,0.0,0.0,0.0,215.250000,1.0,1.750000,0.125,6.750000,21.625000,4.5,158.250000,85.000000,68.750000,215.250000,0.375000,0.250000,0.750000,65.250000,2017.0,2.625000,0.500000,2.625000,1.375000,356.0,1722.0,47.0,1.75,920.0,550.0,33.475959,33.882600,1,1,1,8,67.358559,6,0.0,3,0,7,0.875000,0.875000
2017tur,3.714286,3.285714,0.428571,4.285714,4.428571,0.571429,0.285714,1.857143,0.428571,5.714286,11.142857,0.0,0.0,0.0,8.400000,0.0,8.400000,14.000000,58.400000,36.0,3.0,0.30,6.500000,0.300000,0.0,0.1,56.000000,1.000000,0.800000,1.00,0.600000,1.0,0.0,1.0,0.900000,0.1,0.0,0.1,264.100000,1.0,0.800000,0.500,10.200000,0.100000,3.2,199.200000,96.000000,100.000000,264.100000,0.800000,0.500000,0.700000,-32.300000,2017.0,2.800000,0.600000,3.000000,2.000000,584.0,2641.0,116.0,0.80,1320.0,1000.0,-17.386072,90.605959,0,0,7,10,73.219887,0,0.0,59,0,3,0.300000,0.300000
